In [13]:
library(DBI)
library(RSQLite)

In [14]:
conn = dbConnect(SQLite(), dbname = "c:/RSQL/Customer.sqlite")
# 데이터 베이스를 하나 만들고 싶을 때,
# DBMS는 SQLite()를 이용해
# 어디에? c:/RSQL 폴더 아래에 Customer.sqlite라는 이름으로

# dbConnect(SQLite(), dbname = "c:/RSQL/Customer.sqlite")라는 connector를 conn이라고 저장해준 것

- 만들어 놓기만 한 거고 내용은 아무것도 없다.

In [15]:
# 데이터 베이스와의 연결 해제
# dbDisconnect(conn)

In [16]:
##### PRAGMA statement : SQLite에 특정된 SQL 확장

###  sqlite에서 외래 키를 설정하기 위해서는 다음 문장을 실행해야 함
dbExecute(conn, "PRAGMA foreign_keys=ON")

# Customer_Info 테이블 생성하기
sql = "CREATE TABLE IF NOT EXISTS Customer_Info
					( 	CID TEXT PRIMARY KEY, 
						Name TEXT,
						Address TEXT,
						Tel TEXT,
						BDay TEXT,
						SID TEXT,
						Job TEXT,
						Sex TEXT,
						Married TEXT )"
#   ( 	CID TEXT PRIMARY KEY, 
# 						Name TEXT,
# 						Address TEXT,
# 						Tel TEXT,
# 						BDay TEXT,
# 						SID TEXT,
# 						Job TEXT,
# 						Sex TEXT,
# 						Married TEXT )

# CREATE TABLE IF NOT EXISTS Customer_Info 
# -> Customer_Info라는 TABLE이 없으면 만들어라,있으면 만들지 말고
# 이런 이름(CID, Name, Address, Tel, BDay, SID, ...)들의 필드를 가진, type은 TEXT인.

# 위에서 정의했던 conn이라는 connector이용해서 sqp라는 문장을 실행
rs = dbSendQuery(conn, sql)
# dbSendQuerry라는 함수를 불러서 반환되는 값을
# 굳이 rs라는 변수에 저장하고
# 왜 아래에서 또 rs를 굳이 제거?
# 사실 dbSendQuery(conn, sql) 이것만 실행해도 되긴 함
# 매뉴얼에서 dbSendQuery라는 함수를 보면
# dbSendQuery라는 함수는 S4 object를 반환하는데,,
# 만약 이 반환되는 값을 임의의 변수에 저장하지 않으면
# error는 아니고 warning이 출력됨
# 왜?
# dbSendQuery가 안 닫혀서..
# 그냥 예전에 최귭 교수님이 파일 열었으면 닫아야된다고 했던 거랑
# 같은 원리인 것 같음
# 이것도 dbSendQuery사용했으면 제거해주는 게 그냥 .. 깔끔하다고 생각하면 될 듯

# rs 제거
dbClearResult(rs)

# 테이블이 생성되었는지(존재하는지) 확인
dbExistsTable(conn, "Customer_Info")

[1] 0

[1] TRUE

- TRUE라고 반환됨
  - 만들어졌음을 알 수 있음
  - 내용은 없고 table만 생성된 것임
  - table안에 data 넣고 빼고 혹은 추가 table 생성 등의 작업을 할 수 있음

In [17]:
sql = "CREATE TABLE IF NOT EXISTS Customer_Score  
				( 	CID TEXT, 
					Year TEXT not null,
					On_Score INTEGER default 0,
					Off_Score INTEGER default 0,
					PRIMARY KEY (CID, Year),
					FOREIGN KEY(CID) REFERENCES Customer_Info(CID) )"

rs = dbSendQuery(conn, sql)

# rs 제거
dbClearResult(rs)

# 테이블이 생성되었는지(존재하는지) 확인
dbExistsTable(conn, "Customer_Score")

[1] TRUE

- 2개의 table을 만듦

In [18]:
dbListTables(conn)

[1] "Customer_Info"  "Customer_Score"

In [19]:
# # Customer_Score 테이블 삭제하기
# rs = dbSendQuery(conn, "DROP TABLE Customer_Score")

# # rs 제거
# dbClearResult(rs)

# # 테이블이 존재하는지(삭제되었는지) 확인
# dbExistsTable(conn, "Customer_Score")

---

> 데이터 삽입, 갱신, 삭제

In [20]:
# Customer_Info 테이블에 데이터 insert
sql = " INSERT INTO Customer_Info 
VALUES ('100001', '김민지', '서울시 광진구', '423-8627', 
'1975-07-01', '750701-2534218', '공무원', '1', '0') "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

ERROR: Error: UNIQUE constraint failed: Customer_Info.CID


In [21]:
# Customer_Info 테이블에 데이터 삽입 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Info")
# * => 해당 table에서 모든 field를 불러와라

CID,Name,Address,Tel,BDay,SID,Job,Sex,Married
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
100001,김민지,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0


In [22]:
sql = " INSERT INTO Customer_Score 
VALUES ('100001', '2012', 42, 111) "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

sql = " INSERT INTO Customer_Score 
VALUES ('100001', '2013', 58, 89) "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

# Customer_Score 테이블에 데이터 삽입 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Score ")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>
100001,2012,42,111
100001,2013,58,89


- Customer_Info 테이블에 있는 데이터 update

In [25]:
sql = "UPDATE Customer_Info SET Address = '서울시 영등포구' 
					WHERE CID = '100001' "
# CID = '100001'을 만족하는 고객만 update한다.

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

In [26]:
dbGetQuery(conn, "SELECT * FROM Customer_Info")

CID,Name,Address,Tel,BDay,SID,Job,Sex,Married
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
100001,김민지,서울시 영등포구,423-8627,1975-07-01,750701-2534218,공무원,1,0


---

In [28]:
# Customer_Score 테이블에 있는 데이터 update
sql = "UPDATE Customer_Score SET Off_Score = 98 
WHERE CID = '100001' and Year='2013' "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

# Customer_Score 테이블에 있는 데이터 갱신 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Score ")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>
100001,2012,42,111
100001,2013,58,98


In [29]:
# Customer_Score 테이블에 있는 특정 데이터 delete
sql = "DELETE FROM Customer_Score 
WHERE CID = '100001' and Year='2013' "

rs = dbSendQuery(conn, sql)
dbClearResult(rs)

# Customer_Score 테이블에 있는 특정 데이터 삭제 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Score")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>
100001,2012,42,111


In [30]:
# Customer_Score 테이블에 있는 전체 데이터 delete
sql = "DELETE FROM Customer_Score "
rs = dbSendQuery(conn, sql)
dbClearResult(rs)

# Customer_Score 테이블에 있는 전체 데이터 삭제 결과 확인
dbGetQuery(conn, "SELECT * FROM Customer_Score ")

CID,Year,On_Score,Off_Score
<chr>,<chr>,<int>,<int>


---

> dbWriteTable()를 이용하여 데이터를 한 번에 삽입해보자

In [32]:
# Customer 데이터베이스 연결
conn = dbConnect(SQLite(), dbname = "c:/RSQL/Customer.sqlite")

# Customer 데이터베이스 안에 있는 테이블 리스트 확인하기
dbListTables(conn)

# CSV 화일의 데이터를 Customer_Info 테이블에 삽입하기
customer_data = read.csv('customer_data.csv')
dbWriteTable(conn = conn, name = "Customer_Info", customer_data, overwrite=T, row.names=FALSE)

# Customer_Info 테이블에 있는 데이터 확인하기
dbReadTable(conn, "Customer_Info")

[1] "Customer_Info"  "Customer_Score"

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


- 혹은 아래와 같이 확인할 수 있다. 

In [35]:
dbGetQuery(conn, "SELECT * FROM Customer_Info ")
# 귀찮으니 dbReadTable로 처리하자

CID,Name,Address,Tel,Bday,SID,Job,Sex,Married
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
100001,김민주,서울시 광진구,423-8627,1975-07-01,750701-2534218,공무원,1,0
100002,정영미,경기도 수원시,321-4568,1977-02-01,770201-2834721,회사원,1,1
100003,최소영,제주도 서귀포시,624-2172,1980-01-25,800125-2432712,학생,1,0
100004,박진재,강원도 속초시,543-9467,1960-05-02,600502-1645394,공무원,0,1
100005,이순석,충남 보령시,254-9124,1982-06-01,820601-1543714,회사원,0,0
100006,박선희,서울시 강남구,459-2187,1975-04-05,801220-2487416,회사원,1,0
100007,김화연,경남 경주시,284-8147,1980-01-04,790221-2482217,학생,1,0
100008,박민선,부산시 해운대구,347-9124,1985-04-04,850527-2123827,학생,1,0
100009,노상수,전북 군산시,547-2984,1981-01-30,741025-1441282,공무원,0,1


In [36]:
customer_score = read.csv('customer_score.csv')
dbWriteTable(conn = conn, name = "Customer_Score", customer_score,overwrite=T, row.names=FALSE)

# Customer_Score 테이블에 있는 데이터 확인하기
dbReadTable(conn, "Customer_Score")

CID,Year,On_Score,Off_Score
<int>,<int>,<int>,<int>
100001,2012,42,111
100002,2012,12,30
100003,2012,86,128
100004,2012,0,87
100005,2012,122,56
100006,2012,98,17
100007,2012,0,0
100008,2012,56,172
100009,2012,24,45
